In [2]:
import os
import sys

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore

from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt
# %matplotlib qt
import seaborn as sns

import bk.load
import bk.compute
import bk.plot
import bk.signal


In [3]:
from multiprocessing import Pool

In [4]:
def mod_index(m,n):
    m = m/np.sum(m)

    H = -np.sum(m * np.log(m))
    Dkl = np.log(n) - H

    return Dkl / np.log(n)

In [5]:
def lfp_env(lfp,low,high):
    lfp_filt = bk.signal.passband(lfp,low,high,order = 2)
    return bk.signal.enveloppe(lfp_filt)

def lfp_phase(lfp,low,high):
    lfp_filt = bk.signal.passband(lfp,low,high,order = 2)
    return bk.signal.hilbert(lfp_filt)[1]

In [12]:
def comodulogram(lfp, amp_freq_min, amp_freq_max, phase_freq_min, phase_freq_max, freq_band, phase_nbins):
    bins = np.linspace(-np.pi, np.pi, phase_nbins)
    bins = list(zip(bins, np.roll(bins, -1)))[:-1]
    nbins = len(bins)

    amp_freq_vector = np.arange(amp_freq_min, amp_freq_max, freq_band)
    phase_freq_vector = np.arange(phase_freq_min, phase_freq_max, freq_band)

    amp = []
    for i in tqdm(amp_freq_vector):
        lfp_filt = bk.signal.passband(lfp, i, i+freq_band, order=1)
        amp.append(bk.signal.enveloppe(lfp_filt))

    phase = []
    for i in tqdm(phase_freq_vector):
        lfp_filt = bk.signal.passband(lfp, i, i+freq_band, order=1)
        phase.append(bk.signal.hilbert(lfp_filt)[1])

    comod = np.empty((len(amp), len(phase)))

    print(len(amp))
    for i, a in enumerate(amp):
        for j, p in enumerate(phase):
            m = np.empty(len(bins))
            for k, (l, h) in enumerate(bins):
                m[k] = np.nanmean(a[(l < p.values) & (p.values < h)])
            comod[i, j] = mod_index(m, nbins)

    return amp_freq_vector, phase_freq_vector, comod


In [31]:
bk.load.current_session_linux()
states = bk.load.states()
lfp = bk.load.lfp_in_intervals(73, states['Rem'])


Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713


In [20]:
bk.load.best_channel(6)

73

In [35]:
fa, fp, c = comodulogram(lfp, 2, 300, 2, 30, 1, 18)


100%|██████████| 28/28 [00:30<00:00,  1.07s/it]


298


In [34]:
%matplotlib qt
plt.figure(figsize=(12,12))
plt.imshow(c,origin='lower',extent=[fp[0],fp[-1],fa[0],fa[-1]])

In [10]:
def compute_mod_index(args):
    phase = args[0]
    bins = args[1]
    for j, p in enumerate(phase):
            m = np.empty(len(bins))
            for k, (l, h) in enumerate(bins):
                m[k] = np.nanmean(phase[(l < p.values) & (p.values < h)])
            comod[i, j] = mod_index(m, nbins)


In [11]:
def comodulogram(lfp, amp_freq_min, amp_freq_max, phase_freq_min, phase_freq_max, freq_band, phase_nbins):
    bins = np.linspace(-np.pi, np.pi, phase_nbins)
    bins = list(zip(bins, np.roll(bins, -1)))[:-1]
    nbins = len(bins)

    amp_freq_vector = np.arange(amp_freq_min, amp_freq_max, freq_band)
    phase_freq_vector = np.arange(phase_freq_min, phase_freq_max, freq_band)

    amp = []
    for i in tqdm(amp_freq_vector):
        lfp_filt = bk.signal.passband(lfp, i, i+freq_band, order=1)
        amp.append(bk.signal.enveloppe(lfp_filt))

    phase = []
    for i in tqdm(phase_freq_vector):
        lfp_filt = bk.signal.passband(lfp, i, i+freq_band, order=1)
        phase.append(bk.signal.hilbert(lfp_filt)[1])

    comod = np.empty((len(amp), len(phase)))

    worker = Pool(12)
    worker.map(compute_mod_index, zip(amp, [bins]*len(amp)))

    return amp_freq_vector, phase_freq_vector, c


fa, fp, c = comodulogram(lfp, 2, 125, 2, 20, 2, 18)


100%|██████████| 9/9 [00:09<00:00,  1.02s/it]


AttributeError: 'float' object has no attribute 'values'

In [ ]:
%matplotlib qt
plt.figure(figsize=(12,12))
plt.imshow(c,origin='lower',extent=[fp[0],fp[-1],fa[0],fa[-1]])

NameError: name 'a' is not defined

In [42]:
a = [1,1,1,1,1,1,1,1,1,1,1]
b = [1,2,3]

In [44]:
for i in zip(a,b): print(i)

(1, [1, 2, 3])
